In [3]:
import os
import dask.dataframe as dd
import pandas as pd
import numpy as np

/Users/mammadli/miniconda3/envs/py_jop/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


In [42]:
print("Loading .csv netflow files from 'data/csv/netflow/' and saving as chuncked .parquet files\n")

csv_dir = "data/csv/netflow/"
parquet_dir = "data/parquet/netflow/"

i = 1
for file in os.listdir(csv_dir):
    
    file_name, file_extension = os.path.splitext(file)
    
    if file_extension == ".csv" and not os.path.exists(parquet_dir+"no_ip_port/"+file_name+"_chunks"):
    
        # Load csv file
        print("#"+str(i)+"\nLoading: '"+file+"' ...")
        csv_path = csv_dir+file
        dd_df = dd.read_csv(csv_path)
        
        # Save to parquet file original NetFlow data
        orig_parquet = "original/"+file_name+"_chunks"
        print("Saving original to: '"+parquet_dir+orig_parquet+"' ...")
        dd_df = dd_df.repartition(partition_size="100MB")
        dd_df.to_parquet(parquet_dir+orig_parquet, compression="gzip")
        
        # Save to parquet file NetFlow data with src/dst ip/port
        # Drop duplicates
        parquet_ip_port = "ip_port/"+file_name+"_chunks"
        print("Saving column selection with src/dst ip/port to: '"+parquet_dir+parquet_ip_port+"' ...")
        dd_df = dd_df[['IPV4_SRC_ADDR', 'L4_SRC_PORT', 
                 'IPV4_DST_ADDR', 'L4_DST_PORT',
                'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
                'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label']]
        dd_df = dd_df.drop_duplicates()
        dd_df = dd_df.repartition(partition_size="100MB")
        dd_df.to_parquet(parquet_dir+parquet_ip_port, compression="gzip")

        # Save to parquet file NetFlow data wo src/dst ip/port
        # Drop duplicates
        parquet_no_ip_port = "no_ip_port/"+file_name+"_chunks"
        print("Saving column selection wo src/dst ip/port to: '"+parquet_dir+parquet_no_ip_port+"' ...")
        dd_df = dd_df[['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
                'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label']]
        dd_df = dd_df.drop_duplicates()
        dd_df = dd_df.repartition(partition_size="100MB")
        #print("Shape:" +str(dd_df.shape[0].compute())+", "+str(dd_df.shape[1]))
        dd_df.to_parquet(parquet_dir+parquet_no_ip_port, compression="gzip")
        i += 1

Loading .csv netflow files from 'data/csv/netflow/' and saving as chuncked .parquet files

#1
Loading: NF-BoT-IoT-v2.csv ...
Saving original to: data/parquet/netflow/original/NF-BoT-IoT-v2_chunks ...
Saving column selection with src/dst ip/port to: data/parquet/netflow/ip_port/NF-BoT-IoT-v2_chunks ...
Saving column selection wo src/dst ip/port to: data/parquet/netflow/no_ip_port/NF-BoT-IoT-v2_chunks ...


In [16]:
dd_df = dd.read_parquet("data/parquet/cic/original/CIC-BoT-IoT_chunks_")
dd_df = dd_df.dropna()
                
dd_df = dd_df.repartition(partition_size="100MB")
dd_df.to_parquet("data/parquet/cic/original/CIC-BoT-IoT_chunks", compression="gzip")

In [4]:
import glob
import socket
import struct
import random

https://www.kaggle.com/datasets/vigneshvenkateswaran/bot-iot

In [5]:
i=1
def readDf(f):
    global i
    print(i)
    i += 1
    return pd.read_csv(f)
    

print("Loading .csv flow files from 'data/csv/flow/' and saving as chuncked .parquet files\n")

csv_dir = "data/csv/flow/"
parquet_dir = "data/parquet/flow/"

all_files = glob.glob(csv_dir+"BoT_IoT/*.csv")
df = pd.concat((readDf(f) for f in all_files))

Loading .csv flow files from 'data/csv/flow/' and saving as chuncked .parquet files

1


/var/folders/qk/0rq62snn37d4kk5pqqpg5cj40000gn/T/ipykernel_59725/2430614948.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


2


/var/folders/qk/0rq62snn37d4kk5pqqpg5cj40000gn/T/ipykernel_59725/2430614948.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


3


...

/var/folders/qk/0rq62snn37d4kk5pqqpg5cj40000gn/T/ipykernel_59725/2430614948.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


73


/var/folders/qk/0rq62snn37d4kk5pqqpg5cj40000gn/T/ipykernel_59725/2430614948.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


74


/var/folders/qk/0rq62snn37d4kk5pqqpg5cj40000gn/T/ipykernel_59725/2430614948.py:6: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


In [10]:
df['flgs_number'] = pd.factorize( df['flgs'] )[0]
df['state_number'] = pd.factorize( df['state'] )[0]
df['proto_number'] = pd.factorize( df['proto'] )[0]

In [13]:
df.drop(columns=['subcategory ','pkSeqID','stime','flgs','category','state','proto','seq'], inplace=True)

df['saddr'] = df.saddr.apply(str)
df['sport'] = df.sport.apply(str)
df['daddr'] = df.daddr.apply(str)
df['dport'] = df.dport.apply(str)

df['saddr'] = df.saddr.apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

df['saddr'] = df['saddr'] + ':' + df['sport']
df['daddr'] = df['daddr'] + ':' + df['dport']


df.drop(columns=['sport','dport'],inplace=True)
df.rename(columns={"attack": "label"},inplace = True)
label_ground_truth = df[["saddr", "daddr","label"]]
df = pd.get_dummies(df, columns = ['flgs_number','state_number', 'proto_number'])

df = df.reset_index()
df.replace([np.inf, -np.inf], np.nan,inplace = True)

df.fillna(0, inplace = True)
df.drop(columns=['index'],inplace=True)

In [15]:
df = df.drop_duplicates()

In [18]:
orig_parquet = "original/BoT_IoT_chunks"
print("Saving original to: "+parquet_dir+orig_parquet+" ...")
size99MB = int(df.memory_usage().sum()/1e6/99) + 1
dd_df = dd.from_pandas(df, npartitions=size99MB)
dd_df.to_parquet(parquet_dir+orig_parquet, compression="gzip")


#dd_df = dd_df.repartition(partition_size="100MB")
#df.to_parquet(parquet_dir+orig_parquet, compression="gzip")

Saving original to: data/parquet/flow/original/BoT_IoT_chunks.parquet ...
